In [5]:
import pandas as pd
import json

In [6]:
df = pd.read_csv("characters.csv")

In [7]:
df

,character,Ant-Man,Avengers: Age of Ultron,Avengers: Endgame,Avengers: Infinity War,Black Panther,Captain America: Civil War,Captain America: The First Avenger,Captain America: The Winter Soldier,Captain Marvel,...,Spider-Man: Homecoming,The Avengers,Thor,Thor: Ragnarok,Thor: The Dark World,total lines,total words,movie appearances,avg words per line,avg lines per movie
0,TONY STARK,0,173,137,119,0,165,0,0,0,...,74,152,0,0,0,1788,22590,9,12,198
1,STEVE ROGERS,3,133,119,25,0,154,186,218,3,...,7,121,0,0,0,977,8653,10,8,97
2,THOR,0,55,70,74,0,0,0,0,0,...,0,47,166,289,133,863,9487,7,10,123
3,PETER PARKER,0,0,7,39,0,73,0,0,0,...,402,0,0,0,0,551,5517,4,10,137
4,NATASHA ROMANOFF,1,79,69,15,0,64,0,112,3,...,0,97,0,1,0,498,4689,10,9,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,STRIKE AGENT #3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,1,1,1
648,TALL RAVAGER,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
649,TENT KIDS,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,1,1,1
650,THOMAS RICHARDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [25]:
data = {}

for i in range(len(df)):
    array = []
    for column in df.columns:
        if column == "character":
            data[df.iloc[i]["character"]] = []
        else:
            array.append({column:int(df.iloc[0][column])})
    data[df.iloc[i]["character"]] = array
    


In [ ]:
data

In [28]:
with open('characters.json', 'w') as file:
    json.dump(data, file)